Downloading and unarchiveing the amazon data, it is already split to train and test, we are going to use it for training and validation, we will be testingg with our own data.

In [3]:
!wget "https://s3.amazonaws.com/fast-ai-nlp/amazon_review_polarity_csv.tgz"
!tar -xvf amazon_review_polarity_csv.tgz

--2020-12-05 19:37:54--  https://s3.amazonaws.com/fast-ai-nlp/amazon_review_polarity_csv.tgz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.28.214
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.28.214|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 688339454 (656M) [application/x-tar]
Saving to: ‘amazon_review_polarity_csv.tgz.1’

amazon_review_polar 100%[===================>] 656.45M  27.1MB/s    in 41s     

2020-12-05 19:38:36 (15.9 MB/s) - ‘amazon_review_polarity_csv.tgz.1’ saved [688339454/688339454]

amazon_review_polarity_csv/
amazon_review_polarity_csv/train.csv
amazon_review_polarity_csv/readme.txt
amazon_review_polarity_csv/test.csv


Prepearing the data: the firs column of the dataset is the lable: labble 2 is positive and lable 1 is negative
the second row is the subject, and the third is the full rewiev

In [1]:
import torch
import torch.nn as nn
import torch.optim

from sklearn.model_selection import train_test_split

import os
import numpy as np
import json
import pandas as pd

from tqdm import tqdm

from transformers import BertTokenizer, BertForSequenceClassification

#by loading the data with the right column names we dont have to do much in terms of prepearing
train_data_amazon = pd.read_csv("amazon_review_polarity_csv/train.csv", names=("labels", "subject", "data"))
valid_data_amazon = pd.read_csv("amazon_review_polarity_csv/test.csv", names=("labels", "subject", "data"))

#we will only use a portion of the datasets due to time concerns
train_data = train_data_amazon.head(72000)
valid_data = valid_data_amazon.head(10000)

#we have to subtract 1 from the labels to get labels 0 and 1 instead of 1 and 2
train_data["labels"] -= 1
valid_data["labels"] -= 1

display(train_data)

,labels,subject,data
0,1,Stuning even for the non-gamer,This sound track was beautiful! It paints the ...
1,1,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...
2,1,Amazing!,This soundtrack is my favorite music of all ti...
3,1,Excellent Soundtrack,I truly like this soundtrack and I enjoy video...
4,1,"Remember, Pull Your Jaw Off The Floor After He...","If you've played the game, you know how divine..."
...,...,...,...
71995,1,An Essential for the Musicologist,Though Blacking is sometimes given to high-min...
71996,0,You sent the wrong product,you sent an identidal Flipper to the Right han...
71997,0,Why Is This Lifestyle Glamorized?,"No, I didn't see it. NO ... I wouldn't see it...."
71998,0,Agnes of Gawd,Did this movie have a point? I really don't th...


In [2]:
if not torch.cuda.is_available():
    print("WARNING. GPU is not available. Did you change your runtime to GPU?")

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels = 2)

_ = model.to(device)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [3]:
for p in model.base_model.parameters():
    p.requires_grad = False

In [4]:
# training loop

batch_size = 512
num_epochs = 5
optimizer = torch.optim.Adam(model.parameters())

def training():
    for epoch in range(num_epochs):
        start_positions = list(range(0, len(train_data), batch_size))
        np.random.shuffle(start_positions)

        train_acc = 0
        train_loss = 0
        train_len = 0
        model.train()

        #we will use the tqdm progress bare librarry to better monitor the training
        start_positions = tqdm(start_positions)
        start_positions.set_description(f'Epoch {epoch + 1}/{num_epochs} training ')

        for start in start_positions:
            batch = train_data.iloc[start:start+batch_size]
            optimizer.zero_grad()
            encoded = tokenizer(batch.data.values.tolist(), truncation=True, max_length=128, padding=True, return_tensors='pt')
            input_ids = encoded["input_ids"].to(device)
            attention_mask = encoded["attention_mask"].to(device)
            labels = torch.LongTensor(batch["labels"].tolist()).to(device)

            outputs = model(input_ids, attention_mask = attention_mask, labels = labels)

            loss = outputs[0]
            train_loss += loss.item()

            loss.backward()
            optimizer.step()

            predictions = outputs[1].argmax(axis=1)
            train_acc += torch.eq(predictions, labels).sum()

            train_len += batch_size #we count how many data we trained on to calculate the accuraccy on the fly
            if(train_len > len(train_data)): #the last batch might be smaller so we need to correct for that to get an accurate accuraccy
                train_len = len(train_data)
            start_positions.set_postfix({"loss" : train_loss, "accuraccy" : (train_acc/train_len)}) #we update loss and accuraccy after each batch

        #print(f"valid_loss {train_loss} acc {train_acc / len(train_data)}")

        start_positions = list(range(0, len(valid_data), batch_size))
        valid_acc = 0
        valid_loss = 0
        valid_len = 0
        model.eval()

        #we will use the tqdm progress bare librarry to better monitor the validation
        start_positions = tqdm(start_positions)
        start_positions.set_description(f'Epoch {epoch + 1}/{num_epochs} validating ')
        for start in start_positions:
            batch = valid_data.iloc[start:start+batch_size]
            encoded = tokenizer(batch.data.values.tolist(), truncation=True, max_length=128, padding=True, return_tensors='pt')
            input_ids = encoded["input_ids"].to(device)
            attention_mask = encoded["attention_mask"].to(device)
            labels = torch.LongTensor(batch["labels"].tolist()).to(device)

            outputs = model(input_ids, attention_mask = attention_mask, labels = labels)

            loss = outputs[0]
            valid_loss += loss.item()

            predictions = outputs[1].argmax(axis=1)
            valid_acc += torch.eq(predictions, labels).sum()

            valid_len += batch_size #we count how many data we validated on to calculate the accuraccy on the fly
            if(valid_len > len(valid_data)): #the last batch might be smaller so we need to correct for that to get an accurate accuraccy
                valid_len = len(valid_data)
            start_positions.set_postfix({"loss" : valid_loss, "accuraccy" : (valid_acc/valid_len)}) #we update loss and accuraccy after each batch
        #print(f"valid_loss {valid_loss} acc {valid_acc / len(valid_data)}")

In [5]:
training()

Epoch 5/5 validating : 100%|██████████| 20/20 [00:50<00:00,  2.53s/it, loss=11.5, accuraccy=tensor(0.7269, device='cuda:0')]
